# 本章简介
另一种常用于降维方法就是特征抽取，特征抽取可以将原始数据集变化到一个维度更低的新的特征子空间，从而对数据进行压缩，避免维度灾难  
本章读者将学习三种降维技术：  
+ 无监督数据压缩——主成分分析(PCA)
+ 基于类别可分最大化的监督降维技术——线性判别分析(LDA)
+ 通过核主成分分析进行非线性降维 5  

# 5.1 无监督数据降维技术-主成分分析
> 本章介绍了主成分分析(PCA)的原理、实现和使用方法

### 问题：特征选择和特征抽取有什么区别？
> 特征抽取后的新特征是原来特征的一个映射，而特征选择后的特征是原来特征的一个子集 5  

特征抽取算法会将数据转换或者映射到一个新的特征空间，可以理解为：在尽可能保持相关信息的情况下，对数据进行压缩的一种方法  
特征抽取通常用于提高计算效率，同样也可以帮助我们降低"维度灾难"——尤其当模型不适于正则化处理时  
主成分分析(PCA)是一种广泛应用于不同领域的无监督线性数据转换技术，其突出作用是降维  
简而言之，PCA的目标是在高维数据中找到最大方差的方向，并将数据映射到一个维度不大于原始数据的新的子空间上 5  
如下图所示，新的子空间上正交的坐标轴（主成分）可被解释为方差最大的方向。在此，$x_1$和$x_2$为原始特征的坐标轴，而PC1和PC2即为主成分  
![5-1](../syn_pic/py_machine_learning/5-1.png)
如果使用PCA降维，我们将构建一个$d\times k$维的转换矩阵W，这样就可以将一个样本向量x映射到一个新的k维特征子空间上去，此空间的维度小于原始的d维特征空间：  
$$x=[x_1,x_2,\dots,x_d]，x\in{R}^d$$
$$\downarrow xW,W\in{R}^{d\times k}$$
$$z=[z_1,z_2,\dots,z_k],z\in{R}^k$$
完成从原始d维数据到新的k维子空间(一般情况下$k\ll d$)的转换后，第一主成分的方差应该是最大的，由于各主成分之间是不相关的（正交的），后续各主成分也具备尽可能大的方差  
需要注意的是，主成分的方向对数据值的范围高度敏感，如果特征的值使用不同的度量标准，我们需要先对特征进行标准化处理  
我们先通过以下几个步骤来概括以下算法的流程：
1. 对原始d维数据集做标准化处理 5\*2    